In [1]:
import numpy as np
from collections import deque
from copy import deepcopy
from string import ascii_uppercase

In [2]:
#Check if position is traversable
def good_pos(graph, pos):
    x = pos[0]
    y = pos[1]
    if x < 0 or x >= len(graph):
        return False
    if y < 0 or y >= len(graph[x]):
        return False
    value = graph[x][y]
    return value == '.'

#Breadth-First Search
def BFS(graph, start, end=None, get_path=False, get_dist=False, get_junction=False):
    dx = [-1, 1]
    dy = [-1, 1]
    queue = deque([start])
    dist = {start: 0}
    
    junction = []
    
    ended = False
    while len(queue):
        cur_pos = queue.popleft()
        cur_dist = dist[cur_pos]
        if cur_pos == end:
            ended = True
            break
        adj = 0
        for i in range(0, 2):
            nxt_dist = cur_dist + 1
            #move in x
            nxt_pos = (cur_pos[0]+dx[i], cur_pos[1])
            if good_pos(graph, nxt_pos) :
                if nxt_pos not in dist.keys():
                    queue.append(nxt_pos)
                    dist[nxt_pos] = nxt_dist
                adj += 1
            #move in y
            nxt_pos = (cur_pos[0], cur_pos[1]+dy[i])
            if good_pos(graph, nxt_pos):
                if nxt_pos not in dist.keys():
                    queue.append(nxt_pos)
                    dist[nxt_pos] = nxt_dist
                adj += 1
        if adj > 2 and cur_pos not in junction:
            junction.append(cur_pos)
            
    if get_junction:
        return junction
                
    max_dist = 0
    for key in dist.keys():
        if dist[key] > max_dist:
            max_dist = dist[key]
    
    if get_path and end is not None:
        path = [end]
        test_dist = max_dist
        dxy = [[1,0],[-1,0],[0,1],[0,-1]]
        while path[-1] != start:
            for delta in dxy:
                test = (path[-1][0]+delta[0],path[-1][1]+delta[1])
                if test in dist and dist[test] < test_dist:
                    path.append(test)
                    test_dist -= 1
                    break
        return path
    
    if get_dist:
        return dist, max_dist
    
    if end is not None and ended:
        return max_dist
    elif end is not None and ended == False:
        return np.nan
    
    return max_dist

def dijkstra(graph, start, end):
    nodes = graph.keys()
    visited = {}
    routes = {}
    for node in nodes:
        visited[node] = np.inf
        routes[node] = []
    visited[start] = 0
    routes[start] = [start]
    
    run = True
    while run:
        for vis in visited.keys():
            for key in graph[vis].keys():
                dist = visited[vis] + graph[vis][key]
                rout = deepcopy(routes[vis])
                rout.append(key)
                if np.isfinite(visited[key]) and dist < visited[key]:
                    visited[key] = dist
                    routes[key] = rout
                elif np.isfinite(visited[key]) == False:
                    visited[key] = dist
                    routes[key] = rout
        
        run = False
        for vis in visited.keys():
            if np.isfinite(visited[vis]) == False:
                run = True
                break
    
    return visited[end], routes[end]

In [3]:
data = np.loadtxt('day20_input.txt', comments=None, delimiter='\n', dtype=np.str)

maze = []
for i in range(0, len(data)):
    maze.append(list(data[i]))
maze=np.array(maze)

In [4]:
#Find portals
letters = []
CAPS = ascii_uppercase
for i in range(0, len(maze)):
    for j in range(0, len(maze[i])):
        if maze[i,j] in CAPS:
            letters.append([maze[i,j], [i,j]])

#Find portal entry position
portals = {}
while len(letters) > 0:
    pos = letters[0][1]
    drct = -1
    for i in range(0, len(letters)):
        if letters[i][1] == [pos[0]+1,pos[1]]:
            drct = 0
            break
        elif letters[i][1] == [pos[0],pos[1]+1]:
            drct = 1
            break
    ptl = letters[0][0]+letters[i][0]
    
    letters.pop(i)
    letters.pop(0)
    
    if drct == 0:
        if pos[0]+2 >= len(maze):
            root = [pos[0]-1,pos[1]]
        elif maze[pos[0]+2,pos[1]] == '.':
            root = [pos[0]+2,pos[1]]
        else:
            root = [pos[0]-1,pos[1]]
    elif drct == 1:
        if pos[1]-1 < 0:
            root = [pos[0],pos[1]+2]
        elif pos[1]+2 >= len(maze[0]):
            root = [pos[0],pos[1]-1]
        elif maze[pos[0],pos[1]+2] == '.':
            root = [pos[0],pos[1]+2]
        else:
            root = [pos[0],pos[1]-1]
    
    if ptl in portals.keys():
        portals[ptl+'2'] = tuple(root)
    else:
        portals[ptl] = tuple(root)
        

#Get distances between portals
#and decent
conx = {}
for key in portals.keys():
    connected = {}
    distances, max_dist = BFS(maze, portals[key], get_dist=True)
    for key2 in portals.keys():
        if key == key2:
            continue
        if portals[key2] in distances.keys():
            dist = distances[portals[key2]]
            if len(key2) > 2:
                connected[key2[:2]] = dist
            else:
                connected[key2] = dist
            
    if key[:-1] in conx.keys():
        for key2 in connected:
            conx[key[:-1]][key2] = connected[key2]
    else:
        conx[key] = connected

#Find route using dijkstra
dist, route = dijkstra(conx, 'AA', 'ZZ')
print('Part 1 Solution:', dist+len(route)-2)

Part 1 Solution: 594


# Redo with BFS as Dijkstra is not suited for part 2

In [5]:
def take_portal(graph, cur_pos, nxt_pos, portals, jumps, levels=False):
    nxt_pos = list(nxt_pos)
    x_n = nxt_pos[0]
    y_n = nxt_pos[1]
    x_c = cur_pos[0]
    y_c = cur_pos[1]
    if x_n < 0 or x_n >= len(graph):
        return tuple(nxt_pos)
    if y_n < 0 or y_n >= len(graph[x_n]):
        return tuple(nxt_pos)
    value = graph[x_n][y_n]
    if value in ascii_uppercase:
        ptl = None
        for key in portals:
            if portals[key][0] == x_c and portals[key][1] == y_c:
                ptl = key
                break
                
        if len(ptl) > 2:
            ptl = ptl[:2]
                
        if ptl == 'AA' or ptl == 'ZZ':
            return tuple(nxt_pos)
            
        #Gentry level trying to go up
        if levels and (x_c, y_c) == jumps[ptl][0] and nxt_pos[2] == 0:
            return tuple(nxt_pos)
        #Going up
        elif (x_c, y_c) == jumps[ptl][0]:
            nxt_pos[0] = jumps[ptl][1][0]
            nxt_pos[1] = jumps[ptl][1][1]
            if levels:
                nxt_pos[2] -= 1
        #Going down
        elif (x_c, y_c) == jumps[ptl][1]:
            nxt_pos[0] = jumps[ptl][0][0]
            nxt_pos[1] = jumps[ptl][0][1]
            if levels:
                nxt_pos[2] += 1
    
    return tuple(nxt_pos)

#Breadth-First Search
def BFS_levels(graph, start, end, portals, jumps, levels=False):
    dx = [-1, 1]
    dy = [-1, 1]
    start = (start[0], start[1], 0)
    end = (end[0], end[1], 0)
    queue = deque([start])
    dist = {start: 0}
    
    while len(queue):
        cur_pos = queue.popleft()
        cur_dist = dist[cur_pos]
        if cur_pos == end:
            return cur_dist
        
        for i in range(0, 2):
            nxt_dist = cur_dist + 1
            #move in x
            nxt_pos = (cur_pos[0]+dx[i], cur_pos[1], cur_pos[2])
            nxt_pos = take_portal(graph, cur_pos, nxt_pos, portals, jumps, levels)
            if good_pos(graph, nxt_pos):
                if nxt_pos not in dist.keys():
                    queue.append(nxt_pos)
                    dist[nxt_pos] = nxt_dist
            #move in y
            nxt_pos = (cur_pos[0], cur_pos[1]+dy[i], cur_pos[2])
            nxt_pos = take_portal(graph, cur_pos, nxt_pos, portals, jumps, levels)
            if good_pos(graph, nxt_pos):
                if nxt_pos not in dist.keys():
                    queue.append(nxt_pos)
                    dist[nxt_pos] = nxt_dist
    
    return np.inf

In [6]:
jumps = {} #outer, inner
for key in portals.keys():
    if key == 'AA' or key == 'ZZ':
        continue
    
    if len(key) > 2:
        key2 = key[:2]
    else:
        key2 = key+'2'
    if len(key) > len(key2):
        idx = key2
    else:
        idx = key
        
    if idx in jumps.keys():
        continue
        
    if (portals[key][0] == 2 or portals[key][0] == len(maze)-3) or\
        (portals[key][1] == 2 or portals[key][1] == len(maze[0])-3):
        jumps[idx] = [portals[key], portals[key2]]
    else:
        jumps[idx] = [portals[key2], portals[key]]

In [7]:
dist = BFS_levels(maze, portals['AA'], portals['ZZ'], portals, jumps)
print('Part 1 Solution:', dist)
dist = BFS_levels(maze, portals['AA'], portals['ZZ'], portals, jumps, True)
print('Part 2 Solution:', dist)

Part 1 Solution: 594
Part 2 Solution: 6812
